# ComfyUI Operations Dashboard
Live overview of Drive usage, queue health, manifests, and voice assets. Toggle auto-refresh to keep the panels up-to-date while the worker runs.

In [ ]:
# Summary cards + auto-refresh
import asyncio
import os
import shutil
from datetime import datetime
from pathlib import Path

import ipywidgets as widgets
import pandas as pd
from IPython.display import display

from src.config import PlaygroundConfig
from src.env import detect_gpu
from src.queue import list_items, is_paused

cfg = PlaygroundConfig.load()
cfg.ensure_directories()

summary_box = widgets.HBox()
queue_table = widgets.Output()
manifest_table = widgets.Output()
voice_table = widgets.Output()
status_label = widgets.HTML()

interval_slider = widgets.IntSlider(value=10, min=5, max=60, step=5, description="Seconds")
auto_toggle = widgets.ToggleButton(value=False, description="Auto-refresh", button_style="success")
refresh_button = widgets.Button(description="Refresh now", button_style="info")

_refresh_task = None


def _human_bytes(num):
    for unit in ["B", "KB", "MB", "GB", "TB"]:
        if num < 1024:
            return f"{num:.1f} {unit}"
        num /= 1024
    return f"{num:.1f} PB"


def _summary_cards(metrics):
    cards = []
    for label, value in metrics.items():
        cards.append(
            widgets.HTML(
                value=f"<div style='padding:12px;border-radius:6px;background:#222;color:#fff;min-width:160px;text-align:center;'><div style='font-size:12px;color:#999'>{label}</div><div style='font-size:20px;font-weight:700'>{value}</div></div>"
            )
        )
    return widgets.HBox(cards)


def _gather_metrics():
    disk = shutil.disk_usage(str(cfg.drive_root))
    queue_items = list_items()
    gpu = detect_gpu()
    metrics = {
        "Drive Used": _human_bytes(disk.used),
        "Drive Free": _human_bytes(disk.free),
        "Queue Depth": str(len(queue_items)),
        "GPU": gpu.get("name", "None") if gpu.get("available") == "true" else "CPU",
    }
    return metrics, queue_items


def _render_queue(rows):
    queue_table.clear_output()
    if not rows:
        with queue_table:
            display(widgets.HTML("<b>No queue items.</b>"))
        return
    df = pd.DataFrame(rows)
    df["created_at"] = pd.to_datetime(df["created_at"], unit="s")
    if "updated_at" in df:
        df["updated_at"] = pd.to_datetime(df["updated_at"], unit="s")
    with queue_table:
        display(df)


def _render_manifests():
    manifest_table.clear_output()
    data = []
    for path in sorted(cfg.manifests_dir.glob("*.json")):
        stat = path.stat()
        data.append({
            "name": path.name,
            "size": _human_bytes(stat.st_size),
            "modified": datetime.fromtimestamp(stat.st_mtime),
        })
    df = pd.DataFrame(data) if data else pd.DataFrame(columns=["name", "size", "modified"])
    with manifest_table:
        display(df)


def _render_voice_assets():
    voice_table.clear_output()
    voice_dir = cfg.models_dir / "voice"
    entries = []
    if voice_dir.exists():
        for path in voice_dir.rglob("*.json"):
            entries.append({"asset": path.name, "folder": str(path.parent.relative_to(cfg.models_dir))})
    df = pd.DataFrame(entries) if entries else pd.DataFrame(columns=["asset", "folder"])
    with voice_table:
        display(df)


def refresh(_=None):
    metrics, queue_rows = _gather_metrics()
    summary_box.children = tuple(_summary_cards(metrics).children)
    _render_queue(queue_rows)
    _render_manifests()
    _render_voice_assets()
    status_label.value = f"Last updated: {datetime.utcnow().strftime('%H:%M:%S')} | Queue paused: {is_paused()}"


def _on_toggle(change):
    global _refresh_task
    if change["new"]:
        refresh()
        async def _loop():
            while auto_toggle.value:
                await asyncio.sleep(interval_slider.value)
                refresh()
        try:
            _refresh_task = asyncio.create_task(_loop())
        except RuntimeError:
            loop = asyncio.get_event_loop()
            _refresh_task = loop.create_task(_loop())
    else:
        if _refresh_task:
            _refresh_task.cancel()
            _refresh_task = None


def _on_refresh(_):
    refresh()

refresh_button.on_click(_on_refresh)
auto_toggle.observe(_on_toggle, names="value")

controls = widgets.HBox([refresh_button, auto_toggle, interval_slider])
dashboard = widgets.VBox([
    summary_box,
    controls,
    status_label,
    widgets.Label("Queue"),
    queue_table,
    widgets.Label("Manifests"),
    manifest_table,
    widgets.Label("Voice Assets"),
    voice_table,
])
display(dashboard)
refresh()

In [ ]:
# Queue controls
import ipywidgets as widgets
from IPython.display import display

from src.queue import is_paused, list_items, purge_completed, retry_item, set_paused

status_label = widgets.HTML(value=f"Queue paused: {is_paused()}")
retry_button = widgets.Button(description="Retry failed", button_style="warning")
purge_button = widgets.Button(description="Purge done", button_style="danger")
pause_toggle = widgets.ToggleButton(value=is_paused(), description="Pause queue")


def _retry(_):
    for row in list_items(status="failed"):
        retry_item(row["id"])
    status_label.value = f"Queue paused: {is_paused()}"


def _purge(_):
    purge_completed()
    status_label.value = f"Queue paused: {is_paused()}"


def _pause(change):
    set_paused(change.new)
    status_label.value = f"Queue paused: {is_paused()}"

retry_button.on_click(_retry)
purge_button.on_click(_purge)
pause_toggle.observe(_pause, names="value")

display(widgets.VBox([widgets.HBox([retry_button, purge_button, pause_toggle]), status_label]))

In [ ]:
# Masked token input (store in kernel only)
from getpass import getpass
COMFYUI_API_KEY = getpass('Enter COMFYUI API key (wont be written to Drive):')
print('COMFYUI_API_KEY set in kernel (masked). Use COMFYUI_API_KEY variable in other cells.')

In [ ]:
# ComfyUI monitor
import os
from datetime import datetime

import requests
from IPython.display import JSON

base = os.environ.get("COMFYUI_API_BASE", "http://127.0.0.1:8188").rstrip("/")
health = {}
try:
    r = requests.get(f"{base}/", timeout=5)
    health["status"] = r.status_code
    health["preview"] = r.text[:80]
    meta = requests.get(f"{base}/api/system_stats", timeout=5)
    if meta.ok:
        health.update(meta.json())
except Exception as exc:
    health["error"] = str(exc)

display(JSON(health))

In [ ]:
# One-click Compose & Run test (posts a trivial flow to COMFYUI API)
from src import composer
import requests
base = os.environ.get('COMFYUI_API_BASE','http://127.0.0.1:8188').rstrip('/')
headers = {}
if 'COMFYUI_API_KEY' in globals():
    headers['Authorization'] = 'Bearer ' + globals().get('COMFYUI_API_KEY')
path, flow = composer.compose_flow('dashboard smoke test')
try:
    r = requests.post(base + '/api/flow/run', json={'flow': flow}, headers=headers, timeout=30)
    print('status', r.status_code)
    try:
        print(r.json())
    except:
        print(r.text[:400])
except Exception as e:
    print('Failed to POST to COMFYUI API:', e)

[Back to ComfyUI Playground](comfyui_playground.ipynb)